In [50]:
import pandas as pd
import numpy as np
from keras.layers import Activation, Dense, Dropout, SpatialDropout1D,Input,Masking,Bidirectional, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.models import Sequential, Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from random import seed
#seed(1)
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
import math
from sklearn.metrics import roc_auc_score
import itertools
from sklearn.metrics import classification_report, confusion_matrix
from prettytable import PrettyTable

#defining the early stopping criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,restore_best_weights=True,patience=3)


In [51]:
def split_70(x):
    return int((round((x/15)*0.7))*15)

In [52]:
gs = pd.read_csv('LSTM_1_hour.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)

In [37]:
gs.dischargestatus.unique()

array([1, 0])

In [53]:
cols_to_use = ['uhid','pulserate', 'ecg_resprate',
       'spo2', 'heartrate', 'mean_bp', 'sys_bp', 'dia_bp',
       'peep', 'pip', 'map', 'tidalvol', 'minvol', 'ti', 'fio2',
       'abd_difference_y',
       'currentdateheight',
       'currentdateweight','dischargestatus', 
       'new_ph', 
       'rbs',  'stool_day_total', 
       'temp', 'total_intake', 'totalparenteralvolume',
       'tpn-tfl', 'typevalue_Antibiotics', 'typevalue_Inotropes',
       'urine','gender', 'birthweight',
       'birthlength', 'birthheadcircumference', 'inout_patient_status',
       'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'gestation']

In [54]:
def range_finder(p):
    length = p
    fractional = (p/15.0) - math.floor(p/15.0)
    return int(round(fractional*15))

In [55]:
def randomize(u):
    
    f_df = pd.DataFrame(columns=u.columns)
    for i in u.uhid.unique():
        x = u[u['uhid']==i]
        x = x[range_finder(len(x)):len(x)]

        f_df = f_df.append(x,ignore_index=True)
        
        return f_df

In [56]:
gd = gs[cols_to_use]

In [57]:
final_df = gd.copy()



In [58]:
final_df.fillna(-999,inplace=True)

1320

0

In [59]:
def make_lstm(gd):


    final_df = gd.copy()
    



    final_df.fillna(-999,inplace=True)


    # In[ ]:

    train = final_df[:split_70(len(final_df))]
    test = final_df[split_70(len(final_df)):]
#     train = final_df[:2520]
#     test = final_df[2520:]


    # In[ ]:


    y_train = train['dischargestatus']
    X_train = train.drop('dischargestatus',axis=1)
    X_train = X_train.drop('uhid',axis=1)
    #X_train = X_train.drop('visittime',axis=1)

    y_test = test['dischargestatus']
    X_test = test.drop('dischargestatus',axis=1)
    X_test = X_test.drop('uhid',axis=1)
    #X_test = X_test.drop('startdate',axis=1)


    # In[ ]:

    auc_roc_inter = []
    val_a = []
    train_a = []


    #converting the data into a numpy array
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    ytrain1 = []
    for i in range(0,len(y_train),15):
        #print(i)
        y1 = y_train[i:i+15]
        ytrain1.append(y1[-1])

    ytest1 = []
    for i in range(0,len(y_test),15):
        #print(i)
        y1 = y_test[i:i+15]
        ytest1.append(y1[-1])

    ytrain1 = np.array(ytrain1)
    ytest1 = np.array(ytest1)

    Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
    Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

    return Xtrain,Xtest,ytrain1,ytest1

In [60]:
def lstm_model(Xtrain,Xtest,ytrain1,ytest1):
    auc_roc_inter = []
    val_a = []
    train_a = []
    for i in range(2):
        #Building the LSTM model
        X = Input(shape=(None, 45), name='X')
        mX = Masking()(X)
        lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
        mX = lstm(mX)
        L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
        y = Dense(1, activation="sigmoid")(L)
        outputs = [y]
        inputs = [X]
        model = Model(inputs,outputs)
        model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

        v_a = []
        t_a = []
        #fitting the model
        model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
        #history = model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
        for i in range(len(model.history.history['val_accuracy'])):
            v_a.append(model.history.history['val_accuracy'][i])
            t_a.append(model.history.history['accuracy'][i])
        #predictions
        y_pred = model.predict(Xtest)
        #y_pred = y_pred.round()
        y_test = np.array(ytest1)
        y_pred = np.array(y_pred)
        y_test = pd.DataFrame(y_test)
        y_test = np.array(y_test)

        def acc(x):
            if x>0.5:
                return 1
            else:
                return 0

        y_model=[]
        for i in y_pred:
            y_model.append(acc(i))
        y_answer=[]
        for j in y_test:
            y_answer.append(acc(j))
            
        val_a.append(v_a)
        train_a.append(t_a)
        print("roc_auc_score")
        print(y_answer)
        print(y_pred)
        print(y_test)
        auc_roc_inter.append(roc_auc_score(y_answer,y_pred))
        continue
    
    
        
    return auc_roc_inter,y_model,y_answer

In [61]:
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)

In [62]:
import scipy.stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [63]:
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_a = mean_confidence_interval(an)



Train on 52 samples, validate on 10 samples
Epoch 1/38
52/52 [==============================] - 6s 118ms/step - loss: 0.7009 - accuracy: 0.4231 - val_loss: 0.9770 - val_accuracy: 0.4000
Epoch 2/38
52/52 [==============================] - 1s 13ms/step - loss: 0.5817 - accuracy: 0.7500 - val_loss: 1.2310 - val_accuracy: 0.4000
Epoch 3/38
52/52 [==============================] - 1s 12ms/step - loss: 0.5505 - accuracy: 0.7692 - val_loss: 1.4672 - val_accuracy: 0.4000
Epoch 4/38
52/52 [==============================] - 1s 11ms/step - loss: 0.5112 - accuracy: 0.7692 - val_loss: 1.5412 - val_accuracy: 0.4000
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
roc_auc_score
[0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
[[0.52048504]
 [0.5294354 ]
 [0.7191244 ]
 [0.709629  ]
 [0.709629  ]
 [0.709629  ]
 [0.6922106 ]
 [0.5578411 ]
 [0.5573293 ]
 [0.59421134]
 [0.7021984 ]
 [0.6090121 ]
 [0.5768186 ]
 [0.5885199 ]
 [0.6271951 ]
 [0.6

In [64]:
cm = confusion_matrix(y_answer,y_model)
ppv_1 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_1 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

In [65]:
gs = pd.read_csv('LSTM_6_hour.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)

In [66]:
gd = gs[cols_to_use]

In [67]:
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)


In [68]:
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_6 = mean_confidence_interval(an)


Train on 314 samples, validate on 56 samples
Epoch 1/38
314/314 [==============================] - 16s 52ms/step - loss: 0.6233 - accuracy: 0.6752 - val_loss: 0.5033 - val_accuracy: 0.8214
Epoch 2/38
314/314 [==============================] - 5s 16ms/step - loss: 0.5156 - accuracy: 0.7898 - val_loss: 0.6174 - val_accuracy: 0.4821
Epoch 3/38
314/314 [==============================] - 4s 12ms/step - loss: 0.4810 - accuracy: 0.7771 - val_loss: 0.4254 - val_accuracy: 0.7321
Epoch 4/38
314/314 [==============================] - 4s 13ms/step - loss: 0.4336 - accuracy: 0.8153 - val_loss: 0.4129 - val_accuracy: 0.7321
Epoch 5/38
314/314 [==============================] - 5s 15ms/step - loss: 0.4051 - accuracy: 0.8185 - val_loss: 0.7845 - val_accuracy: 0.5179
Epoch 6/38
314/314 [==============================] - 5s 17ms/step - loss: 0.3847 - accuracy: 0.8344 - val_loss: 0.6339 - val_accuracy: 0.6607
Epoch 7/38
314/314 [==============================] - 4s 13ms/step - loss: 0.3471 - accuracy: 0.

In [69]:
cm = confusion_matrix(y_answer,y_model)
ppv_6 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_6 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

In [70]:
gs = pd.read_csv('LSTM_12_hour.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_12 = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_12 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_12 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

Train on 628 samples, validate on 111 samples
Epoch 1/38
628/628 [==============================] - 15s 23ms/step - loss: 0.5358 - accuracy: 0.7197 - val_loss: 1.2418 - val_accuracy: 0.1081
Epoch 2/38
628/628 [==============================] - 8s 12ms/step - loss: 0.3537 - accuracy: 0.8806 - val_loss: 2.1377 - val_accuracy: 0.2793
Epoch 3/38
628/628 [==============================] - 7s 12ms/step - loss: 0.2425 - accuracy: 0.9045 - val_loss: 2.8185 - val_accuracy: 0.2793
Epoch 4/38
628/628 [==============================] - 8s 13ms/step - loss: 0.1904 - accuracy: 0.9140 - val_loss: 2.9582 - val_accuracy: 0.0811
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
roc_auc_score
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

628/628 [==============================] - 7s 11ms/step - loss: 0.1963 - accuracy: 0.9220 - val_loss: 3.0165 - val_accuracy: 0.0360
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
roc_auc_score
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [71]:
gs = pd.read_csv('LSTM_48_hour.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_48 = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_48 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_48 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

Train on 2427 samples, validate on 429 samples
Epoch 1/38
2427/2427 [==============================] - 31s 13ms/step - loss: 0.4391 - accuracy: 0.7998 - val_loss: 0.8913 - val_accuracy: 0.5944
Epoch 2/38
2427/2427 [==============================] - 27s 11ms/step - loss: 0.2941 - accuracy: 0.8681 - val_loss: 1.0066 - val_accuracy: 0.4848
Epoch 3/38
2427/2427 [==============================] - 24s 10ms/step - loss: 0.2927 - accuracy: 0.8739 - val_loss: 0.6804 - val_accuracy: 0.4755
Epoch 4/38
2427/2427 [==============================] - 25s 10ms/step - loss: 0.2906 - accuracy: 0.8752 - val_loss: 1.0734 - val_accuracy: 0.4918
Epoch 5/38
2427/2427 [==============================] - 27s 11ms/step - loss: 0.2385 - accuracy: 0.9081 - val_loss: 1.8004 - val_accuracy: 0.4918
Epoch 6/38
2427/2427 [==============================] - 29s 12ms/step - loss: 0.2418 - accuracy: 0.9056 - val_loss: 1.4338 - val_accuracy: 0.4685
Restoring model weights from the end of the best epoch
Epoch 00006: early sto

In [72]:
l = [["1 Hour" ,c_a, ppv_1,npv_1],["6 Hour" ,c_6, ppv_6,npv_6],["12 Hour" ,c_12, ppv_12,npv_12],["48 Hour" ,c_48, ppv_48,npv_48]]
table = PrettyTable(['Parameter', 'AUC-ROC', 'PPV','NPV'])

for rec in l:
    table.add_row(rec)

print(table)

+-----------+----------------------------------------------------------------+------+------+
| Parameter |                            AUC-ROC                             | PPV  | NPV  |
+-----------+----------------------------------------------------------------+------+------+
|   1 Hour  |       (0.778125, 0.3413492121851469, 1.214900787814853)        | 0.1  | 1.0  |
|   6 Hour  | (0.43103070175438596, -1.1335600788391715, 1.9956214823479435) | 0.0  | 0.82 |
|  12 Hour  |  (0.4641504329004329, -1.7267760493853284, 2.655076915186194)  | 0.0  | 0.83 |
|  48 Hour  | (0.3598975466008772, 0.10977026380618665, 0.6100248293955677)  | 0.05 | 0.62 |
+-----------+----------------------------------------------------------------+------+------+


In [73]:
gs = pd.read_csv('LSTM_1_week.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_1w = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_1w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_1w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

Train on 7619 samples, validate on 1345 samples
Epoch 1/38
7619/7619 [==============================] - 78s 10ms/step - loss: 0.3770 - accuracy: 0.8342 - val_loss: 0.0835 - val_accuracy: 0.9978
Epoch 2/38
7619/7619 [==============================] - 78s 10ms/step - loss: 0.3073 - accuracy: 0.8697 - val_loss: 0.0801 - val_accuracy: 1.0000
Epoch 3/38
7619/7619 [==============================] - 89s 12ms/step - loss: 0.2918 - accuracy: 0.8758 - val_loss: 0.0667 - val_accuracy: 1.0000
Epoch 4/38
7619/7619 [==============================] - 82s 11ms/step - loss: 0.3007 - accuracy: 0.8743 - val_loss: 0.0912 - val_accuracy: 0.9933
Epoch 5/38
7619/7619 [==============================] - 88s 12ms/step - loss: 0.2764 - accuracy: 0.8884 - val_loss: 0.0469 - val_accuracy: 1.0000
Epoch 6/38
7619/7619 [==============================] - 91s 12ms/step - loss: 0.2790 - accuracy: 0.8854 - val_loss: 0.1126 - val_accuracy: 0.9859
Epoch 7/38
7619/7619 [==============================] - 75s 10ms/step - loss

Train on 7619 samples, validate on 1345 samples
Epoch 1/38
7619/7619 [==============================] - 70s 9ms/step - loss: 0.3799 - accuracy: 0.8270 - val_loss: 0.1160 - val_accuracy: 1.0000
Epoch 2/38
7619/7619 [==============================] - 67s 9ms/step - loss: 0.3263 - accuracy: 0.8590 - val_loss: 0.0897 - val_accuracy: 1.0000
Epoch 3/38
7619/7619 [==============================] - 79s 10ms/step - loss: 0.2932 - accuracy: 0.8770 - val_loss: 0.0752 - val_accuracy: 1.0000
Epoch 4/38
7619/7619 [==============================] - 91s 12ms/step - loss: 0.3000 - accuracy: 0.8732 - val_loss: 0.0939 - val_accuracy: 1.0000
Epoch 5/38
7619/7619 [==============================] - 79s 10ms/step - loss: 0.2954 - accuracy: 0.8783 - val_loss: 0.0516 - val_accuracy: 0.9948
Epoch 6/38
7619/7619 [==============================] - 69s 9ms/step - loss: 0.2865 - accuracy: 0.8820 - val_loss: 0.0813 - val_accuracy: 1.0000
Epoch 7/38
7619/7619 [==============================] - 63s 8ms/step - loss: 0.

In [74]:
gs = pd.read_csv('LSTM_2_week.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_2w = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_2w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_2w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

Train on 14445 samples, validate on 2550 samples
Epoch 1/38
14445/14445 [==============================] - 127s 9ms/step - loss: 0.3224 - accuracy: 0.8582 - val_loss: 2.6029 - val_accuracy: 0.1118
Epoch 2/38
14445/14445 [==============================] - 133s 9ms/step - loss: 0.2472 - accuracy: 0.9003 - val_loss: 2.2481 - val_accuracy: 0.1510
Epoch 3/38
14445/14445 [==============================] - 123s 9ms/step - loss: 0.2468 - accuracy: 0.8956 - val_loss: 2.6263 - val_accuracy: 0.2145
Epoch 4/38
14445/14445 [==============================] - 124s 9ms/step - loss: 0.2527 - accuracy: 0.8948 - val_loss: 1.9069 - val_accuracy: 0.1816
Epoch 5/38
14445/14445 [==============================] - 125s 9ms/step - loss: 0.2507 - accuracy: 0.8931 - val_loss: 2.2885 - val_accuracy: 0.4322
Epoch 6/38
14445/14445 [==============================] - 115s 8ms/step - loss: 0.2382 - accuracy: 0.9012 - val_loss: 2.3669 - val_accuracy: 0.3376
Epoch 7/38
14445/14445 [==============================] - 113s 

Train on 14445 samples, validate on 2550 samples
Epoch 1/38
14445/14445 [==============================] - 118s 8ms/step - loss: 0.3187 - accuracy: 0.8611 - val_loss: 2.3430 - val_accuracy: 0.2106
Epoch 2/38
14445/14445 [==============================] - 133s 9ms/step - loss: 0.2637 - accuracy: 0.8903 - val_loss: 2.0652 - val_accuracy: 0.2200
Epoch 3/38
14445/14445 [==============================] - 157s 11ms/step - loss: 0.2364 - accuracy: 0.9049 - val_loss: 2.7920 - val_accuracy: 0.2122
Epoch 4/38
14445/14445 [==============================] - 179s 12ms/step - loss: 0.2289 - accuracy: 0.9072 - val_loss: 3.5559 - val_accuracy: 0.1145
Epoch 5/38
14445/14445 [==============================] - 235s 16ms/step - loss: 0.2416 - accuracy: 0.9020 - val_loss: 2.8806 - val_accuracy: 0.1039
Restoring model weights from the end of the best epoch
Epoch 00005: early stopping
roc_auc_score
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [76]:
l = [["1 Hour" ,c_a, ppv_1,npv_1],["6 Hour" ,c_6, ppv_6,npv_6],["12 Hour" ,c_12, ppv_12,npv_12],["48 Hour" ,c_48, ppv_48,npv_48],["1 Week" ,c_1w, ppv_1w,npv_1w],["2 Week" ,c_2w, ppv_2w,npv_2w]]
table = PrettyTable(['Parameter', 'AUC-ROC', 'PPV','NPV'])

for rec in l:
    table.add_row(rec)

print(table)

+-----------+----------------------------------------------------------------+------+------+
| Parameter |                            AUC-ROC                             | PPV  | NPV  |
+-----------+----------------------------------------------------------------+------+------+
|   1 Hour  |       (0.778125, 0.3413492121851469, 1.214900787814853)        | 0.1  | 1.0  |
|   6 Hour  | (0.43103070175438596, -1.1335600788391715, 1.9956214823479435) | 0.0  | 0.82 |
|  12 Hour  |  (0.4641504329004329, -1.7267760493853284, 2.655076915186194)  | 0.0  | 0.83 |
|  48 Hour  | (0.3598975466008772, 0.10977026380618665, 0.6100248293955677)  | 0.05 | 0.62 |
|   1 Week  | (0.39566225402130406, 0.33842487349400413, 0.452899634548604)  | 0.35 | 0.32 |
|   2 Week  | (0.4809669950548834, -1.2735240305826347, 2.2354580206924015)  | 0.73 | 0.2  |
+-----------+----------------------------------------------------------------+------+------+


In [77]:
gs = pd.read_csv('LSTM_3_week.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_3w = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_3w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_3w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

Train on 20242 samples, validate on 3573 samples
Epoch 1/38
20242/20242 [==============================] - 244s 12ms/step - loss: 0.3133 - accuracy: 0.8629 - val_loss: 0.4360 - val_accuracy: 0.7128
Epoch 2/38
20242/20242 [==============================] - 176s 9ms/step - loss: 0.2859 - accuracy: 0.8797 - val_loss: 0.6978 - val_accuracy: 0.6152
Epoch 3/38
20242/20242 [==============================] - 163s 8ms/step - loss: 0.2773 - accuracy: 0.8865 - val_loss: 0.4823 - val_accuracy: 0.6860
Epoch 4/38
20242/20242 [==============================] - 168s 8ms/step - loss: 0.2709 - accuracy: 0.8890 - val_loss: 0.0875 - val_accuracy: 0.9978
Epoch 5/38
20242/20242 [==============================] - 180s 9ms/step - loss: 0.2425 - accuracy: 0.9028 - val_loss: 0.3828 - val_accuracy: 0.8701
Epoch 6/38
20242/20242 [==============================] - 167s 8ms/step - loss: 0.2467 - accuracy: 0.9009 - val_loss: 0.5965 - val_accuracy: 0.6888
Epoch 7/38
20242/20242 [==============================] - 164s

Train on 20242 samples, validate on 3573 samples
Epoch 1/38
20242/20242 [==============================] - 163s 8ms/step - loss: 0.3151 - accuracy: 0.8603 - val_loss: 0.2936 - val_accuracy: 0.9317
Epoch 2/38
20242/20242 [==============================] - 181s 9ms/step - loss: 0.2794 - accuracy: 0.8823 - val_loss: 0.3416 - val_accuracy: 0.8455
Epoch 3/38
20242/20242 [==============================] - 167s 8ms/step - loss: 0.2708 - accuracy: 0.8850 - val_loss: 0.4988 - val_accuracy: 0.6896
Epoch 4/38
20242/20242 [==============================] - 187s 9ms/step - loss: 0.2663 - accuracy: 0.8931 - val_loss: 1.0252 - val_accuracy: 0.5908
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
roc_auc_score
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [78]:
gs = pd.read_csv('LSTM_4_week.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_4w = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_4w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_4w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

Train on 24348 samples, validate on 4297 samples
Epoch 1/38
24348/24348 [==============================] - 233s 10ms/step - loss: 0.2243 - accuracy: 0.9069 - val_loss: 1.9079 - val_accuracy: 0.5104
Epoch 2/38
24348/24348 [==============================] - 206s 8ms/step - loss: 0.1738 - accuracy: 0.9330 - val_loss: 1.3923 - val_accuracy: 0.5111
Epoch 3/38
24348/24348 [==============================] - 200s 8ms/step - loss: 0.1593 - accuracy: 0.9394 - val_loss: 1.3711 - val_accuracy: 0.5145
Epoch 4/38
24348/24348 [==============================] - 191s 8ms/step - loss: 0.1522 - accuracy: 0.9409 - val_loss: 1.2306 - val_accuracy: 0.5104
Epoch 5/38
24348/24348 [==============================] - 204s 8ms/step - loss: 0.1513 - accuracy: 0.9423 - val_loss: 1.8064 - val_accuracy: 0.5104
Epoch 6/38
24348/24348 [==============================] - 207s 8ms/step - loss: 0.1423 - accuracy: 0.9485 - val_loss: 0.3776 - val_accuracy: 0.8047
Epoch 7/38
24348/24348 [==============================] - 198s

Train on 24348 samples, validate on 4297 samples
Epoch 1/38
24348/24348 [==============================] - 200s 8ms/step - loss: 0.2266 - accuracy: 0.9039 - val_loss: 1.8349 - val_accuracy: 0.5104
Epoch 2/38
24348/24348 [==============================] - 187s 8ms/step - loss: 0.1733 - accuracy: 0.9321 - val_loss: 0.3101 - val_accuracy: 0.8736
Epoch 3/38
24348/24348 [==============================] - 194s 8ms/step - loss: 0.1586 - accuracy: 0.9393 - val_loss: 1.0744 - val_accuracy: 0.5150
Epoch 4/38
24348/24348 [==============================] - 199s 8ms/step - loss: 0.1503 - accuracy: 0.9441 - val_loss: 2.0101 - val_accuracy: 0.5104
Epoch 5/38
24348/24348 [==============================] - 210s 9ms/step - loss: 0.1393 - accuracy: 0.9491 - val_loss: 0.7237 - val_accuracy: 0.5504
Restoring model weights from the end of the best epoch
Epoch 00005: early stopping
roc_auc_score
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [79]:
l = [["1 Hour" ,c_a, ppv_1,npv_1],["6 Hour" ,c_6, ppv_6,npv_6],["12 Hour" ,c_12, ppv_12,npv_12],["48 Hour" ,c_48, ppv_48,npv_48],["1 Week" ,c_1w, ppv_1w,npv_1w],["2 Week" ,c_2w, ppv_2w,npv_2w],["3 Week" ,c_3w, ppv_3w,npv_3w],["4 Week" ,c_4w, ppv_4w,npv_4w]]

table = PrettyTable(['Parameter', 'AUC-ROC', 'PPV','NPV'])

for rec in l:
    table.add_row(rec)

print(table)

+-----------+----------------------------------------------------------------+------+------+
| Parameter |                            AUC-ROC                             | PPV  | NPV  |
+-----------+----------------------------------------------------------------+------+------+
|   1 Hour  |       (0.778125, 0.3413492121851469, 1.214900787814853)        | 0.1  | 1.0  |
|   6 Hour  | (0.43103070175438596, -1.1335600788391715, 1.9956214823479435) | 0.0  | 0.82 |
|  12 Hour  |  (0.4641504329004329, -1.7267760493853284, 2.655076915186194)  | 0.0  | 0.83 |
|  48 Hour  | (0.3598975466008772, 0.10977026380618665, 0.6100248293955677)  | 0.05 | 0.62 |
|   1 Week  | (0.39566225402130406, 0.33842487349400413, 0.452899634548604)  | 0.35 | 0.32 |
|   2 Week  | (0.4809669950548834, -1.2735240305826347, 2.2354580206924015)  | 0.73 | 0.2  |
|   3 Week  | (0.3684265807017251, 0.25882315810205053, 0.4780300033013997)  | 0.5  | 0.42 |
|   4 Week  |  (0.8560282137273256, 0.40467671246283826, 1.30737971499